# 3. Building and evaluating different models

### How to do this:
- Only use test data set

Models to build:
- a dummy model: mean of y
- a baseline model: using only gender? 
- full model:
    - a simple linear regression
    - a non-parametric linear regression KNN
    - a simple decision tree

All? or only the full models should be trained using CV and should be heavily regularized (_standardize_ first?). 
All models should be compared using the same error metric: mean squarred error 

choosing the best model, validate it, and then test it:
- investigate interpretability, feature importance, etc. 


In [ ]:
# important relevant packages 

from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error # MSE?? rmse? 
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

## random forrest og feature importance 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
#from xgboost import XGBRegressor

import pickle as pkl




: 

In [ ]:
# load data
X_train_embedded = np.load('data/train_embedded.npy', allow_pickle = True)

In [ ]:
# first we want to exclude columns which are not relevant as predictors

X_train_w_pca_vecs[4,0:9]
X_train_gender = X_train_w_pca_vecs[:,0]
X_train_predictors = X_train_w_pca_vecs[:,3:]

X_train_predictors = np.hstack((X_train_gender.reshape(-1,1), X_train_predictors))

X_train_predictors

In [ ]:
# save new df
#np.save("data/train_embedded", X_train_predictors, allow_pickle=True)
# reload 
#test = np.load("data/train_embedded.npy", allow_pickle=True)

## Standardizing embedded features

In [ ]:
# standardize embedded data

scaler = StandardScaler()
X_train_pred_scaled = scaler.fit_transform(X_train_predictors[:,3:])

In [ ]:
non_scaled_cols = X_train_predictors[:,:3]
#X_train_pred_scaled.shape

In [ ]:
X_train_predictors_scaled = np.hstack((non_scaled_cols, X_train_pred_scaled))

## Building models

In [ ]:
performances = []

In [ ]:
y_train.shape
X_train_predictors.shape

In [ ]:
### Dummy model  -  mean of y

mean_value = y_train.mean()
model_name = 'dummy'
performance = np.sqrt(mean_squared_error(y_train, [mean_value]*y_train.shape[0]))
r2 = r2_score(y_train, [mean_value]*y_train.shape[0])
performances.append({'model': model_name,
                     'split': 'train',
                     'rmse': performance.round(4),
                     'r2': r2.round(4)})



In [ ]:
### Baseline model - only predictor is gender - linear regression

gender = X_train_predictors[:,0].reshape(-1,1)
#gender = gender.reshape(-1,1)
reg = LinearRegression().fit(gender, y_train)

preds =  reg.predict(gender)
r2 = r2_score(y_train, preds)
performance = np.sqrt(mean_squared_error(y_train, preds))
performances.append({'model': 'linear-gender',
                         'rmse': performance.round(4),
                         'r2': r2.round(4)})


In [ ]:
### full linear model 
# cv? - this is not working out 
# regularization? 

reg = LinearRegression().fit(X_train_predictors_scaled, y_train)

preds =  reg.predict(X_train_predictors_scaled)
r2 = r2_score(y_train, preds)
performance = np.sqrt(mean_squared_error(y_train, preds))
performances.append({'model': 'linear-full-scaled',
                         'rmse': performance.round(4),
                         'r2': r2.round(4)})


In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression

# Initialize the model
model = LinearRegression()

# Define the cross-validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=1)  # 5-fold cross-validation

# Perform cross-validation with RMSE as the scoring metric
scores = cross_val_score(model, X_train_predictors_scaled, y_train, cv=kf, scoring='neg_root_mean_squared_error')

# Calculate the average RMSE and standard deviation
rmse_scores = -scores  # Convert negative RMSE scores to positive
#print(f"Mean RMSE: {rmse_scores:.2f}") #, Standard Deviation of RMSE: {std_rmse:.2f}")



In [ ]:
model_fit = model.fit(X_train_predictors_scaled, y_train)

preds =  model_fit.predict(X_train_predictors_scaled)
r2 = r2_score(y_train, preds)
performance = np.sqrt(mean_squared_error(y_train, preds))
performances.append({'model': 'linear-full-scaled-cv',
                         'rmse': performance.round(4),
                         'r2': r2.round(4)})

In [ ]:
performances

In [ ]:
### full model but using polynomial linear regression
from sklearn.preprocessing import PolynomialFeatures

X_train_copy = X_train_predictors_scaled.copy()

poly = PolynomialFeatures(2)

X_train_copy = poly.fit_transform(X_train_copy)
reg = LinearRegression().fit(X_train_copy, y_train)
preds = reg.predict(X_train_copy)
r2 = r2_score(y_train, preds)
performance = np.sqrt(mean_squared_error(y_train, preds))
performances.append({'model': 'poly-allpreds-3',
                    'split': 'train',
                    'rmse': performance.round(4),
                    'r2': r2.round(4)})
       

In [ ]:
# full model but using KNN linear regression

for k in [3,5,10,20,30]:
    neigh = KNeighborsRegressor(n_neighbors=k)
    neigh.fit(X_train_predictors, y_train)
    #pkl.dump(neigh, file=open(f'example-models/knn-allpreds-{k}.pkl', 'wb')) # save the model

    preds = neigh.predict(X_train_predictors)

    r2 = r2_score(y_train, preds)
    performance = np.sqrt(mean_squared_error(y_train, preds))
    performances.append({'model': f'knn-allpreds-{k}',
                            'split': 'train',
                            'rmse': performance.round(4),
                            'r2': r2.round(4)})

In [ ]:
k = 100
neigh = KNeighborsRegressor(n_neighbors=k)
neigh.fit(X_train_predictors, y_train)
#pkl.dump(neigh, file=open(f'example-models/knn-allpreds-{k}.pkl', 'wb')) # save the model

preds = neigh.predict(X_train_predictors)

r2 = r2_score(y_train, preds)
performance = np.sqrt(mean_squared_error(y_train, preds))
performances.append({'model': f'knn-allpreds-{k}',
                        'split': 'train',
                        'rmse': performance.round(4),
                        'r2': r2.round(4)})

In [ ]:
performances

In [ ]:
rfreg = RandomForestRegressor(random_state=42) # first, we instantiate the estimator


In [ ]:

param_grid = { 
    'n_estimators': [10, 20, 100, 200, 500],
    'max_depth' : [2, 3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'max_features': [0.3, 0.6, 0.9], # can you guess what this is, without looking at the documentation?
    'ccp_alpha': [0.01, 0.1, 1.0]
}

In [ ]:
# cv
cv_rfr = RandomizedSearchCV(estimator=rfreg, # I am choosing RandomizedSearchCV for speed, but you can also go for GridSearchCV :)
                            param_distributions=param_grid,
                            scoring='neg_mean_squared_error', # this is "neg" because CV wants a metric to maximize
                            n_iter=20, # this should more likely be above 100, and in general the higher the better
                            cv=5)
cv_rfr.fit(X_train_predictors_scaled, y_train)

# 100 min - save model in variable as with pickle 

In [ ]:
cv_rfr.best_estimator_.get_params()

In [ ]:
random_forrest_model = cv_rfr.fit(X_train_predictors_scaled, y_train)

In [ ]:
pkl.dump(random_forrest_model, file=open(f'models/random_forrest.pkl', 'wb')) # save the model

In [ ]:
model = cv_rfr.best_estimator_

preds = model.predict(X_train_predictors_scaled)
r2 = r2_score(y_train, preds)
performance = np.sqrt(mean_squared_error(y_train, preds))
performances.append({'model': 'random_forrest',
                         'split': 'train',
                         'rmse': performance.round(4),
                         'r2': r2.round(4)})

In [ ]:
perf_df = pd.DataFrame(performances)
sns.scatterplot(data=perf_df.sort_values(by='rmse', ascending=False), 
                y='model', 
                x='rmse', 
                marker='s', 
                hue='split', palette=['darkorange', 'grey', 'darkred'])
plt.show()

In [ ]:
importances = cv_rfr.best_estimator_.feature_importances_
# the above computes the (normalized) total reduction of the criterion brought by that feature.

In [ ]:
sns.barplot(x=train.columns.tolist(), y=importances, color=sns.color_palette()[0])
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
#plt.gca().set_xticks(plt.gca().get_xticks()[::2])  # Show every 10th tick # [::10]


plt.show()